In [60]:
 import json
import requests
import string
from collections import Counter
import time


ACCESSTOKEN  = 'EAAEXVBeJumcBADJoWXBQrGsJZCaCnfoOIaxzHssVxjJVJUZC2TCCfzHsH99EZBSg3Y9HMQKXjizPO4k0WVskfAm9t2TnjZCBJdg734PJJfLKD5BBCrNGJO0RLMbhEr4ZANOzwOvzy47XJ1eqHC1mkLLpYbmddXlZB8R7EF8kJjmAZDZD'
PostUrl ='https://graph.facebook.com/v2.5/15925638948/posts?limit=100&fields=attachments,message,created_time,shares&access_token='+ACCESSTOKEN

res = requests.get(PostUrl)

jd = json.loads(res.text)

PostID= jd['data'][0]['id']
print jd['data'][0]['shares']['count']
len(jd['data'])




98


100

In [3]:
#粉絲抓取

PostUrl ='https://graph.facebook.com/v2.5/15925638948?fields=likes&access_token='+ACCESSTOKEN

res = requests.get(PostUrl)

jd = json.loads(res.text)

Fans= jd['likes']

print Fans
def inserFans():
    command="INSERT INTO FBFans (Crawler_Time,Fans) VALUES (?,?)"

    cursor.execute(command,(Crawler_Time,Fans))
    conn.commit()


6635899


In [4]:
#判斷起始NO
#IDrow

import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
cursor = conn.cursor()
cursor.execute("select * from FBPostData   order by PostNO asc")

def Second():
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
    cursor = conn.cursor()
    cursor.execute("select * from FBPostData   order by PostNO asc")
    global rows
    rows = cursor.fetchall()
    global IDrows
    
    IDrows=[]
#自動判斷1 或是第二次以上NO   
    try:
        PostNO= rows[-1][0]+1
        for row in rows:
            IDrows.append( row[-1])
        return PostNO
    except:
        return 1
Second()

1

In [5]:
#偵測PO文NO

def PostNOdetect(id):
    
    
    for row in rows:
        if row[-1]==id:
            NO=row[0]
            
    return NO

#PostNOdetect('15925638948_10154486947508949')

In [6]:
#PO文 抓入SQL

def insertSQL(id,PostNO):
    status='new'
    
    #如果是舊的PO文
    if(id in IDrows ):
        status='old'
        PostNO=PostNOdetect(id)
    
    if(status=='new'):
        command="INSERT INTO FBPostData (PostNO,Created_Time,Message,ID,AttatchTitle,AttatchMessage,Crawler_Time) VALUES (?,?,?,?,?,?,?)"

        cursor.execute(command,(PostNO,created_time,Postmessage,id,attatchTitle,attatchMessage,Crawler_Time))
        
    command="INSERT INTO FBPostChange (PostNO,LikeCount,CommentCount,ShareCount,Crawler_Time) VALUES (?,?,?,?,?)"

    cursor.execute(command,(PostNO,LikeCount,CommentCount,ShareCount,Crawler_Time))
    
# rows = cursor.fetchall()
# for row in rows:

In [7]:
#判斷起始NO



def CommentSecond(PostNO):
    global CommentIDrows
    global Commentrows
    CommentIDrows=[]
    Commentrows=[]
    
    cursor.execute("select * from FBCommentData where PostNO=?  order by CommentNO asc",PostNO)
    Commentrows = cursor.fetchall()
    try:
        CommentNO= Commentrows[-1][1]+1
        for row in Commentrows:
            CommentIDrows.append( row[-3])

        return CommentNO
    except:
        return 1
CommentSecond(3)


1

In [8]:
def CommentNOdetect(id):
    for row in Commentrows:
        if row[-3]==id:
            CommentNO=row[1]
    return CommentNO

#CommentNOdetect('10154486947508949_10154487300663949')

In [9]:
def Comment(PostID):
    start =time.time()
    
    CommentUrl ='https://graph.facebook.com/v2.5/'+PostID+'/comments?fields=like_count,message,comment_count,from,id&limit=200&access_token='+ACCESSTOKEN

    CommentRes = requests.get(CommentUrl)

    CommentJd = json.loads(CommentRes.text)
    
    
    global like_count
    global message
    global comment_count
    global Name
    global UserID
    global CommentID
    global CommentNO
    global CommentStatus
    
    CommentStatus='new'
    CommentNO=CommentSecond(PostNO)
    
    #CommentNO=1
    
    try:
        bug=CommentJd['paging']
    except:
        print PostID
        return 0
    end =time.time()
    time1= end-start
    HowMuch=str(len(CommentJd['data']))
    print '評論載入成功:'+HowMuch+'筆 '+str(time1)+'秒'
    
    
   
    
    start =time.time()
    
    while 'next' in CommentJd['paging'] :
        
        
        for data in CommentJd['data']:
            like_count= data['like_count']
            message=data['message']
            comment_count=data['comment_count']

            Name= data['from']['name']
            UserID= data['from']['id']
            CommentID=data['id']
            
                
            #插入    
            InsertComment(CommentID,CommentNO)
            
            if(CommentID==CommentJd['data'][-1]['id']):

                CommentUrl=CommentJd['paging']['next']
                time.sleep(1)
                CommentRes=requests.get(CommentUrl)
                CommentJd = json.loads(CommentRes.text)

            
            CommentNO+=1
            
    for data in CommentJd['data']:
        like_count= data['like_count']
        message=data['message']
        comment_count=data['comment_count']

        Name= data['from']['name']
        UserID= data['from']['id']
        CommentID=data['id']
        #插入
        InsertComment(CommentID,CommentNO)

        CommentNO+=1
        
    end=time.time()
    time1=end-start
    print '資料輸入完畢:'+str(time1)+'秒'
    
        
    return CommentNO-1
    
    
    
# Comment('15925638948_10154512960348949')    
    
    

In [10]:
def InsertComment(CommentID,CommentNO):
    CommentStatus='new'
    if(CommentID in CommentIDrows ):
        CommentStatus='old'
        CommentNO=CommentNOdetect(CommentID)
    
    if(CommentStatus=='new'):
        command="INSERT INTO FBCommentData (PostNO,CommentNO,Name,Message,CommentID,UserID,Crawler_Time) VALUES (?,?,?,?,?,?,?)"

        cursor.execute(command,(PostNO,CommentNO,Name,message,CommentID,UserID,Crawler_Time))

    
    command="INSERT INTO FBCommentChange (PostNO,CommentNO,LikeCount,ReplyCount,Crawler_Time) VALUES (?,?,?,?,?)"

    cursor.execute(command,(PostNO,CommentNO,like_count,comment_count,Crawler_Time))


In [11]:
#判斷起始NO



def LikeSecond(PostNO):
    global LikeIDrows
    global Likerows
    LikeIDrows=[]
    Likerows=[]
    
    cursor.execute("select * from FBPostLikeChange where PostNO=?  order by LikeNO asc",PostNO)
    Likerows = cursor.fetchall()
    try:
        LikeNO= Likerows[-1][1]+1
        for row in Likerows:
            LikeIDrows.append( row[2])

        return LikeNO
    except:
        return 1
LikeSecond(1)


3

In [12]:
def LikeNOdetect(id):
    for row in Likerows:
        if row[2]==id:
            LikeNO=row[1]
    return LikeNO

#CommentNOdetect('10154486947508949_10154487300663949')

In [34]:
def InsertLike(UserID,LikeNO):
    
    LikeStatus='new'
    if(UserID in LikeIDrows ):
        LikeStatus='old'
#         LikeNO=LikeNOdetect(UserID)
    
    if LikeStatus=='new':
        
        Values.append((PostNO,LikeNO,UserID,UserName,Crawler_Time))
        
        

In [58]:
def Like(PostID):
    global LikeNO
    global UserName
    global UserID
    global CommentStatus
    global Values
    Values=[]
    command="INSERT  INTO FBPostLikeChange (PostNO,LikeNO,UserID,UserName,Crawler_Time) VALUES (?,?,?,?,?)"
    
    start=time.time()
    CommentStatus='new'
    
    LikeNO= LikeSecond(PostNO)
    
    LikeUrl ='https://graph.facebook.com/v2.5/'+PostID+'/likes?pretty=0&limit=1000&access_token='+ACCESSTOKEN

    LikeRes = requests.get(LikeUrl)

    LikeJd = json.loads(LikeRes.text)
    end =time.time()
    time1= end-start
    HowMuch=str(len(LikeJd['data']))
    print '讚數載入成功:'+HowMuch+'個 '+str(time1)+'秒'
    
    start=time.time()
    try:
        while 'next' in LikeJd['paging']:
            for data in LikeJd['data']:
                UserName= data['name']
                UserID= data['id']
                InsertLike(UserID,LikeNO)
                if(UserID== LikeJd['data'][-1]['id']):
                    LikeUrl=LikeJd['paging']['next']
                    #time.sleep(0.1)
                    LikeRes=requests.get(LikeUrl)
                    LikeJd = json.loads(LikeRes.text)
                LikeNO+=1
    except:
        return 0
            
        
    for data in LikeJd['data']:
        UserName= data['name']
        UserID= data['id']
        InsertLike(UserID,LikeNO)
        LikeNO+=1
        
  
    cursor.executemany(command, Values)
    conn.commit()
    end =time.time()
    time1= end-start
    HowMuch=str(len(LikeJd['data']))
    print '資料輸入成功:'+HowMuch+'個 '+str(time1)+'秒'
    
    return LikeNO-1
    
    

In [14]:
cursor.execute("select * from FBPostData   order by PostNO asc")
rows = cursor.fetchall()
print rows[0][0]


1


In [94]:
PostUrl =('https://graph.facebook.com/v2.7/15925638948_10154512299018949/comments?fields=id,from,message,'
          'comment_count,comments.limit(50){from,message,like_count},'
          'like_count,likes.limit(1000){username,id}&limit=100&access_token='+ACCESSTOKEN)
res = requests.get(PostUrl)
jd = json.loads(res.text)
print jd['data'][1]['comment_count']
print len(jd['data'][1]['comments']['data'])

24
21


In [96]:
len(jd['data'][4]['likes']['data'])

274

In [97]:
PostUrl =('https://graph.facebook.com/v2.7/15925638948_10154512960348949?fields=comments.limit(200)&access_token='+ACCESSTOKEN)
res = requests.get(PostUrl)
jd = json.loads(res.text)
print len(jd['comments']['data'])

150


In [ ]:
#開爬

import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
cursor = conn.cursor()


#狀態初始化
end=0
status='new'
IDrows=[]
global PostNO

PostNO=Second()


#粉絲

PostUrl ='https://graph.facebook.com/v2.5/15925638948?fields=likes&access_token='+ACCESSTOKEN
res = requests.get(PostUrl)
jd = json.loads(res.text)
Fans= jd['likes']


#PO文
PostUrl ='https://graph.facebook.com/v2.5/15925638948/posts?limit=100&fields=attachments,message,created_time,shares&access_token='+ACCESSTOKEN
res = requests.get(PostUrl)
jd = json.loads(res.text)




#時間
Crawler_Time = raw_input("Crawler_Time:")
print Crawler_Time

# inserFans()



while( 'next' in jd['paging'] ):
    status='new'
    if end==1:
        break
    
    for post in jd['data']:
        attatchTitle=""
        attatchMessage=""
        CommentCount=0
        LikeCount=0
        ShareCount=0
        Postmessage=""
        
        #例外
        if('message'in post):
            Postmessage=post['message']
        if('shares'in post):
            ShareCount=post['shares']['count']
        created_time=post['created_time']
            
        id=post['id']

        
        #attachments
        if('attachments' in post):
            if('title' in post['attachments']['data'][0]):
                attatchTitle=post['attachments']['data'][0]['title']
            if('description' in post['attachments']['data'][0]):
                attatchMessage=post['attachments']['data'][0]['description']

        if(id in IDrows ):
            status='old'
            PostNO=PostNOdetect(id)
        CommentCount=0
        CommentCount=Comment(id)
        LikeCount=0
        LikeCount=Like(id)
        
#         print CommentCount
        insertSQL(id,PostNO)
        
        if (id==jd['data'][-1]['id']):
            url=jd['paging']['next']

            res=requests.get(url)
            jd = json.loads(res.text)
        
        PostNO+=1

  
    
        
        #停止點
        if (id=='15925638948_10154512135443949'):
            end=1
            
            break
            
    
conn.commit()
print "end"
cursor.close()
conn.close()   

Crawler_Time:20160729
20160729
評論載入成功:26筆 0.732000112534秒
資料輸入完畢:0.53200006485秒
讚數載入成功:427個 0.511999845505秒
資料輸入成功:427個 9.49300003052秒
評論載入成功:32筆 0.348000049591秒
資料輸入完畢:0.644000053406秒
讚數載入成功:190個 0.353999853134秒
資料輸入成功:190個 3.66799998283秒
評論載入成功:12筆 0.398999929428秒
資料輸入完畢:0.237999916077秒
讚數載入成功:243個 0.351999998093秒
資料輸入成功:243個 5.92100000381秒
評論載入成功:2筆 0.306999921799秒
資料輸入完畢:0.039999961853秒
讚數載入成功:81個 0.301000118256秒
資料輸入成功:81個 1.52599978447秒
評論載入成功:200筆 0.733000040054秒
資料輸入完畢:10.4429998398秒
讚數載入成功:1000個 0.927999973297秒


In [62]:
cursor.close()
conn.close()  

In [45]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
cursor = conn.cursor()
query = "INSERT INTO FBPostLikeChange (PostNO,LikeNO,UserID,UserName,Crawler_Time) VALUES (?,?,?,?,?)"
values = []

values.append((1, 2, 3, 4,5))
values.append((2, 2, 3, 4,5))
cursor.executemany(query, values)
conn.commit()
print 'end'
cursor.close()
conn.close()  

end


In [57]:


conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
cursor = conn.cursor()
query = "INSERT INTO FBPostLikeChange (PostNO,LikeNO,UserID,UserName,Crawler_Time) VALUES (?,?,?,?,?)"
values = []
start=time.time()
print '陣列讀取中'
for i in range(0,1000):
    values.append((i, 0, 0, 0,0))
print '陣列讀取完畢'
print '資料輸入中'
cursor.executemany(query, values)
conn.commit()
end =time.time()
time1= end-start

print '資料輸入完必'
print time1
cursor.close()
conn.close()  

陣列讀取中
陣列讀取完畢
資料輸入中
資料輸入完必
18.8329999447


In [56]:
import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_fanpage;UID=sa;PWD=scucc')
cursor = conn.cursor()
   
sql_posts="INSERT INTO post(postNum,fanpageNum,postId,postMessage,postTime,postStory) VALUES (?,?,?,?,?,?)"
print '資料輸入中'
start=time.time()
for n in range(0,1000):
    cursor.execute(sql_posts,(n,n,n,n,n,n))    
    

conn.commit() 
end =time.time()
time1= end-start
print time1
print '資料輸入完畢'

cursor.close()
conn.close()
print "end"

資料輸入中
19.2949998379
資料輸入完畢
end
